### Importações

In [1]:
import pandas as pd; pd.set_option('display.max_columns', None)
from datetime import date, timedelta
import os
from pycaret.classification import *
from scripts.PreparaDataset import *
from scripts.TeoriaRetornos import *

### Funções

In [2]:
def atualiza_dia(_jogos):
    _jogos = prepara_df(_jogos)

    _jogos = _jogos.reset_index(drop=True)

    return _jogos

### Pegando os jogos do dia

In [3]:
today = date.today()
tomorrow = today + timedelta(days=1)

day = today

In [4]:
data_var = day.strftime('%Y-%m-%d')
df_today = pd.read_csv(f'jogos_do_dia/{data_var}.csv')

### Atualizando os datasets que tem jogo hoje

In [5]:
temporada = pd.read_csv('data/Entire_Season.csv')
temporada = temporada[temporada['Date'] < data_var]
temporada = pd.concat([temporada, df_today])
# ligas = temporada[temporada['Date'] == data_var]['League'].unique().tolist()
# temporada = temporada[temporada['League'].isin(ligas)]
temporada.reset_index(drop=True, inplace=True)

### Cria variáveis na temporada e nos jogos do dia

In [6]:
games = atualiza_dia(temporada)
temporada = games[games['Date'] < data_var]
os.makedirs(f'apostas_do_dia/{data_var}', exist_ok=True)

In [7]:
jogos = games[games['Date'] == data_var]

jogos = jogos.drop(columns=['Home_Pts', 'Away_Pts',
       'Back_Home', 'PL_Home', 'Back_Away',
       'PL_Away', 'Back_Over', 'PL_Over', 'PL_Under', 'Back_HA_H', 'PL_HA_H',
       'PL_HA_A', ])

jogos.to_csv(f'jogos_processados/{data_var}.csv', index=False)

jogos[['Date', 'League', 'Time', 'Home', 'Away', 
       'Odds_H', 'Odds_A', 'Over_Line', 'Odds_Over', 'Odds_Under', 'HA_Line',
       'HA_Odds_H', 'HA_Odds_A', ]]

,Date,League,Time,Home,Away,Odds_H,Odds_A,Over_Line,Odds_Over,Odds_Under,HA_Line,HA_Odds_H,HA_Odds_A
5879,2023-11-09,Brasil Nbb,19:00,Cerrado,Caxias do Sul,1.87,1.89,147.5,1.85,1.87,-1.5,1.99,1.73
5880,2023-11-09,Brasil Nbb,19:30,Bauru,Corinthians Paulista,1.66,2.18,151.5,1.89,1.87,-3.5,2.00,1.72
5881,2023-11-09,Brasil Nbb,19:30,Brasília,União Corinthians,1.62,2.26,159.5,1.88,1.86,-4.5,2.08,1.67
5882,2023-11-09,Brasil Nbb,20:00,Pinheiros,Paulistano,2.09,1.71,149.5,1.81,1.85,1.5,1.91,1.80
5883,2023-11-09,Eua Nba,00:00,Denver Nuggets,Golden State Warriors,1.61,2.54,225.5,1.82,2.11,-4.5,2.06,1.87
...,...,...,...,...,...,...,...,...,...,...,...,...,...
5966,2023-11-09,Reino Unido Bbl,16:45,Sheffield Sharks,Surrey Scorchers,1.21,4.29,157.5,1.80,1.86,-11.5,2.09,1.66
5967,2023-11-09,Rússia Liga Vtb United,11:00,Astana,Lokomotiv Kuban,6.45,1.13,160.5,1.87,1.89,11.5,2.04,1.71
5968,2023-11-09,Rússia Liga Vtb United,13:00,Unics Kazan,Runa Basket Moscow,1.13,6.40,163.5,1.88,1.90,-14.5,2.04,1.71
5969,2023-11-09,Rússia Liga Vtb United,13:30,MBA Moscow,Enisey,1.65,2.29,157.5,1.81,1.94,-4.5,2.01,1.77


## Apostas do dia

### Over v0 (LDA)

In [8]:
# Duplicando o dataset
df_over_v0 = jogos.copy()

df_over_v0.reset_index(drop=True, inplace=True)
df_over_v0.replace([np.inf, -np.inf], 0, inplace=True)

if not df_over_v0.empty:
    mdl_over_v0 = load_model('ML/over/v1/v1_over', verbose=False)
    prev = predict_model(mdl_over_v0, df_over_v0)
    prev = prev[prev['prediction_label'] ==  1]
    
    # Filtros pós modelo

    if not prev.empty:
        prev.to_csv(f'apostas_do_dia/{data_var}/over_v0_{data_var}.csv', index=False)

        print('------------ MÉTODO OVER V0 (LDA) ------------')
        print()

        prev['Bet'] = 'Over'
        ligs = prev['League'].unique().tolist()

        display(prev[['Time', 'League', 'Home', 'Away', 'Over_Line', 'Odds_Over', 'prediction_score', 'Bet']])
        print()
    else: print('Sem jogos')
else: print('Sem jogos')

------------ MÉTODO OVER V0 (LDA) ------------



,Time,League,Home,Away,Over_Line,Odds_Over,prediction_score,Bet
4,00:00,Eua Nba,Denver Nuggets,Golden State Warriors,225.5,1.82,1.0000,Over
5,00:00,Eua Nba,Sacramento Kings,Portland Trail Blazers,220.5,1.86,1.0000,Over
6,21:00,Eua Nba,Indiana Pacers,Milwaukee Bucks,239.5,1.83,0.9961,Over
7,23:30,Eua Nba,Orlando Magic,Atlanta Hawks,229.5,1.83,0.9964,Over
8,14:15,Europa Euroliga,Olympiakos,Baskonia,158.5,1.80,1.0000,Over
